In [7]:
from tqdm import tqdm
import xgboost as xgb
import lightgbm as lgb
import catboost

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [8]:
DATA_PATH = "../data" # 학습에 사용할 csv 파일이 저장된 폴더입니다.
TRAIN_FILE = "train.csv" # 학습 및 예측에 사용할 파일입니다.
TRAIN_PATH = os.path.join(DATA_PATH, TRAIN_FILE)
data = pd.read_csv(TRAIN_PATH)

df_brand = pd.read_csv(os.path.join(DATA_PATH, 'brand_keyword_cnt.csv'))
df_info = pd.read_csv(os.path.join(DATA_PATH, 'product_info.csv'))
df_sales = pd.read_csv(os.path.join(DATA_PATH, 'sales.csv'))
df_smp = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))
df_data = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))

In [9]:
data = pd.read_csv(TRAIN_PATH)

timesize = 60

timecol1 = []
for times in range(0, timesize):
    colname='d'+str(-timesize+times)
    timecol1.append(colname)

timecol = []
for times in range(0, timesize):
    colname='d'+str(-timesize+times)
    timecol.append(colname)
timecol.append("target")

In [10]:
DATA_LENGTH, b=data.shape
seed = 1994

FINAL_RESULTS = []

for PRODUCT in tqdm(range(0, DATA_LENGTH)):
    FIRST = data.iloc[PRODUCT:PRODUCT+1, 6:6+timesize+1]
    FIRST.index = [data.columns[6+timesize]]
    FIRST.columns = timecol

    for i in range(1, 465-timesize-6):
        ADD = data.iloc[PRODUCT:PRODUCT+1, 6+i:6+timesize+i+1]
        ADD.index = [data.columns[7+timesize+i]]
        ADD.columns = timecol
        FIRST = pd.concat([FIRST, ADD])
        # FIRST = FIRST.append(ADD, ignore_index=False)

    X_train = FIRST.drop(columns="target")
    y_train = FIRST['target']

    model = XGBRegressor(random_state=seed)
    # model = LGBMRegressor(random_state=seed)
    # model = CatBoostRegressor(random_state=seed, verbose=0)

    try:
        model.fit(X_train, y_train)

        test_df = data.iloc[PRODUCT:PRODUCT+1, 6+i+1:]
        test_df.columns = timecol1

        results = []

        y_pred = model.predict(test_df)
        results.append(round(y_pred[0]))
        for PREDICT_TIME in range(0, 20):
            test_df = test_df.iloc[:, 1:]
            test_df["add"] = round(y_pred[0])
            test_df.columns = X_train.columns
            y_pred = model.predict(test_df)
            results.append(round(y_pred[0]))
    except:
        results = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

    FINAL_RESULTS.append(results)
    
predict_df = pd.concat(FINAL_RESULTS, ignore_index=True)
predict_df


100%|██████████| 28894/28894 [40:31<00:00, 11.88it/s]  


TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [ ]:
# concat 데이터 csv파일로 저장하기
predict_df.to_csv('../submit/submission4.csv', header=True, index=False)

In [11]:
EXAMPLE_FILE = "sample_submission.csv"
EXAMPLE_PATH = os.path.join(DATA_PATH, EXAMPLE_FILE)

submission = pd.read_csv(EXAMPLE_PATH)
submit = pd.DataFrame(FINAL_RESULTS)
submit["ID"] = submit.index
submit = submit[["ID", 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]]
submit.columns = submission.columns

In [12]:
SAVE_PATH = "../submit" 
SUBMISSION_FILE = "submission4.csv"
SUBMIT_PATH = os.path.join(SAVE_PATH, SUBMISSION_FILE)
submit.to_csv(SUBMIT_PATH, index=False)